# Using _egobox_ optimizer _Egor_

## Installation

In [ ]:
%pip install egobox

We import _egobox_ as _egx_ for short.

In [2]:
import numpy as np
import egobox as egx

You may setup the logging level to get optimization progress during the execution

In [3]:
# To display optimization information (none by default)
# import logging
# logging.basicConfig(level=logging.INFO)

## Example 1 : Continuous optimization

### Test functions

In [4]:
xspecs_xsinx = egx.to_specs([[0., 25.]])
n_cstr_xsinx = 0

def xsinx(x: np.ndarray) -> np.ndarray:
    x = np.atleast_2d(x)
    y = (x - 3.5) * np.sin((x - 3.5) / (np.pi))
    return y

In [5]:
xspecs_g24 = egx.to_specs([[0., 3.], [0., 4.]])
n_cstr_g24 = 2

# Objective
def G24(point):
    """
    Function g24
    1 global optimum y_opt = -5.5080 at x_opt =(2.3295, 3.1785)
    """
    p = np.atleast_2d(point)
    return - p[:, 0] - p[:, 1]

# Constraints < 0
def G24_c1(point):
    p = np.atleast_2d(point)
    return (- 2.0 * p[:, 0] ** 4.0
            + 8.0 * p[:, 0] ** 3.0 
            - 8.0 * p[:, 0] ** 2.0 
            + p[:, 1] - 2.0)

def G24_c2(point):
    p = np.atleast_2d(point)
    return (-4.0 * p[:, 0] ** 4.0
            + 32.0 * p[:, 0] ** 3.0
            - 88.0 * p[:, 0] ** 2.0
            + 96.0 * p[:, 0]
            + p[:, 1] - 36.0)

# Grouped evaluation
def g24(point):
    p = np.atleast_2d(point)
    return np.array([G24(p), G24_c1(p), G24_c2(p)]).T


### Continuous optimization with _Egor_

In [6]:
egor = egx.Egor(g24, xspecs_g24, 
                     n_doe=10, 
                     n_cstr=n_cstr_g24, 
                     cstr_tol=1e-3,
                     infill_strategy=egx.InfillStrategy.WB2,
                     target=-5.5,
                     # outdir="./out",
                     # hot_start=True
                    )  # see help(egor) for options

# Specify regression and/or correlation models used to build the surrogates of objective and constraints
#egor = egx.Egor(g24, xlimits_g24, n_cstr=n_cstr_g24, n_doe=10,
#                      regr_spec=egx.RegressionSpec.LINEAR,
#                      corr_spec=egx.CorrelationSpec.MATERN32 | egx.CorrelationSpec.MATERN52)  

In [7]:
res = egor.minimize(n_iter=30)
print(f"Optimization f={res.y_opt} at {res.x_opt}")
print("Optimization history: ")
print(f"Inputs = {res.x_hist}")
print(f"Outputs = {res.y_hist}")

Optimization f=[-5.50864553e+00 -2.26609824e-04  9.78978461e-04] at [2.3296139  3.17903163]
Optimization history: 
Inputs = [[2.2536597  0.5876315 ]
 [2.71826982 1.67190984]
 [0.94027828 2.12879929]
 [0.56626734 1.02369743]
 [2.47701863 3.40141852]
 [1.32080744 0.02945779]
 [0.75348073 3.15480028]
 [0.065796   1.53902157]
 [1.53401084 2.7754243 ]
 [1.86878639 3.79047574]
 [3.         3.99999995]
 [2.35737496 3.18796368]
 [2.32976462 3.17868711]
 [2.3295701  3.17982145]
 [2.3296139  3.17903163]]
Outputs = [[-2.84129120e+00 -2.06596487e+00 -2.91418276e+00]
 [-4.39017966e+00 -7.95222146e+00  7.34543294e-01]
 [-3.06907757e+00 -1.85695942e+00  2.06827333e+00]
 [-1.58996477e+00 -2.29458779e+00 -3.43337794e+00]
 [-5.87843715e+00 -1.39086321e+00  1.01468259e+00]
 [-1.35026523e+00 -3.58005655e+00 -1.13132205e+00]
 [-3.90828100e+00 -6.09499131e-01  1.92797691e+00]
 [-1.60481757e+00 -4.93370110e-01 -2.85164850e+01]
 [-4.30943515e+00 -2.46546469e-01  3.23982098e-01]
 [-5.65926212e+00  1.67021964e+

## Example 2 : Mixed integer optimization

### Test function

In [8]:
xspecs_mixint_xsinx = [egx.XSpec(egx.XType(egx.XType.INT), [0, 25])]
n_cstr_mixint_xsinx = 0

def mixint_xsinx(x: np.ndarray) -> np.ndarray:
    x = np.atleast_2d(x)
    if (np.abs(np.linalg.norm(np.floor(x))-np.linalg.norm(x))< 1e-8):
        y = (x - 3.5) * np.sin((x - 3.5) / (np.pi))
    else:
        raise ValueError(f"Bad input: mixint_xsinx accepts integer only, got {x}")
    return y

### Mixed-integer optimization with _Egor_

In [9]:
egor = egx.Egor(mixint_xsinx, xspecs_mixint_xsinx, 
                     n_doe=3, 
                     infill_strategy=egx.InfillStrategy.EI,
                     target=-15.12,
                    )  # see help(egor) for options
res = egor.minimize(n_iter=30)
print(f"Optimization f={res.y_opt} at {res.x_opt}")
print("Optimization history: ")
print(f"Inputs = {res.x_hist}")
print(f"Outputs = {res.y_hist}")

Optimization f=[-15.12161154] at [19.]
Optimization history: 
Inputs = [[ 9.]
 [ 7.]
 [24.]
 [ 6.]
 [ 5.]
 [ 3.]
 [18.]
 [19.]]
Outputs = [[  5.41123083]
 [  3.14127616]
 [  4.91604976]
 [  1.78601478]
 [  0.68929352]
 [  0.07924194]
 [-14.43198471]
 [-15.12161154]]


## Usage

In [10]:
help(egor)

Help on Egor in module builtins object:

class Egor(object)
 |  Egor(fun, n_cstr=0, cstr_tol=1e-06, n_start=20, n_doe=0, regression_spec=7, correlation_spec=15, infill_strategy=1, q_points=1, par_infill_strategy=1, infill_optimizer=1, n_clusters=1)
 |  
 |  Optimizer constructor
 |  
 |  fun: array[n, nx]) -> array[n, ny]
 |       the function to be minimized
 |       fun(x) = [obj(x), cstr_1(x), ... cstr_k(x)] where
 |          obj is the objective function [n, nx] -> [n, 1]
 |          cstr_i is the ith constraint function [n, nx] -> [n, 1]
 |          an k the number of constraints (n_cstr)
 |          hence ny = 1 (obj) + k (cstrs)
 |       cstr functions are expected be negative (<=0) at the optimum.
 |  
 |   n_cstr (int):
 |       the number of constraint functions.
 |  
 |   cstr_tol (float):
 |       tolerance on constraints violation (cstr < tol).
 |  
 |   xspecs (list(XSpec)) where XSpec(xtype=FLOAT|INT, xlimits=[lower bound, upper bound]):
 |       Bounds of the nx compone